In [1]:
# Required Imports
import boto3

In [13]:
# Bedrock
from langchain_community.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain_aws import BedrockLLM

# prompt and chain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Pdf Loader
from langchain_community.document_loaders import PyPDFLoader

# import FAISS
from langchain_community.vectorstores import FAISS

ModuleNotFoundError: No module named 'langchain_aws'

In [3]:
# creating bedrock client
bedrock_client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")
# creating embedding client
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

In [4]:
# creating llm client
def get_llm():
    llm=Bedrock(model_id="anthropic.claude-v2", client=bedrock_client,model_kwargs={'max_tokens_to_sample': 512})
    return llm

In [5]:
def get_response(llm,vectorstore, question ):
    # create prompt / template
    prompt_template = """
    Human: Please use the given context to provide concise answer to the question
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    Question: {question}

    Assistant:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="similarity", search_kwargs={"k": 5}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
    answer=qa({"query":question})
    return answer['result']

In [6]:
faiss_index = FAISS.load_local(
        index_name="faiss_vdb",
        folder_path = "./",
        embeddings=bedrock_embeddings,
        allow_dangerous_deserialization=True
    )

In [7]:
llm = get_llm()

/Users/omkale/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


In [10]:
query = "Total products in Pure Fresh?"
docs = faiss_index.similarity_search(query)
print(docs)

[Document(page_content="Logistics and Distribution:\nOur Team:\nCustomer Satisfaction:\nSustainability:\nLooking Ahead:\nAs Pure Fresh looks to expand, we are seeking partnerships and opportunities to bring our fresh, high-quality dairy\nproducts to a wider audience. W e are excited about the potential for growth while staying true to our mission and\nvalues. 🚀\nContact Information:\nHow Does Pure Fresh Operate?\nPure Fresh sources milk from farmers and creates products, which they distribute in the market. Let's understand how\nthe business flows in brief.\nFirstly, the farmers bring the milk to collection centers. At the centers, the center owners test the quality of the milk.\nThe milk data is fed into an app by the center owners. Independent contractors then collect milk from centers and\ntake it to chilling centers. At the chilling center, milk from various collection centers is aggregated, and the quality of", metadata={'source': 'sample.pdf', 'page': 1}), Document(page_content="

In [11]:
question = "How many products does Pure Fresh offer?"
get_response(llm, faiss_index, question)

/Users/omkale/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' Based on the given context, Pure Fresh currently offers the following dairy products:\n\n- Fresh Milk\n- Skimmed Milk  \n- Flavored Milk\n- Cottage Cheese\n- Yogurt\n\nSo Pure Fresh offers 5 products.'

In [12]:
question = "who will win elections 2024"
get_response(llm, faiss_index, question)

" Based on the given context, there is no information provided about elections in 2024 or who might win them. The context discusses a dairy company called Pure Fresh and their data analysis needs, but does not mention anything about future elections. So I don't have enough information to determine who might win elections in 2024."